In [1]:
import pandas as pd

df = pd.DataFrame()

In [2]:
import os

In [3]:
list_file = list(os.walk('./vehicle'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
list_file.sort()
    
len(list_file)

771

In [4]:
from tqdm import tqdm
import json
import re

In [5]:
for a in tqdm(range(len(list_file)), desc='Progress', ncols=77):
    with open('./vehicle/' + list_file[a], 'r', encoding='UTF-8') as file:
        resp = file.read()
        
    # = = = = = = = = = = = = = = =
    
    json_data = json.loads(resp)
    
    # = = = = = = = = = = = = = = =
    
    dict_vehicle = {}
    for vehicle in json_data:
        year_loss = []
        
        if vehicle['buildYears'] == None:
            year_loss += [vehicle['subModel']]
        else:
            model_year = vehicle['subModel'].strip()[:-1].strip() if vehicle['subModel'].strip().endswith('-') else vehicle['subModel'].strip()
            list_year = model_year[-4:] if re.search('[0-9]{4} - [0-9]{4}$', model_year) == None else re.search('[0-9]{4} - [0-9]{4}$', model_year).group()
            model = model_year.replace(list_year, '').strip()
            engine = vehicle['vehicle'].strip()

            if model not in dict_vehicle:
                if '-' in list_year:
                    dict_vehicle[model] = {engine: [int(year.strip()) for year in list_year.split('-')]}
                else:
                    dict_vehicle[model] = {engine: [int(list_year)]}
            else:
                if engine not in dict_vehicle[model]:
                    if '-' in list_year:
                        dict_vehicle[model][engine] = [int(year.strip()) for year in list_year.split('-')]
                    else:
                        dict_vehicle[model][engine] = [int(list_year)]
                else:
                    if '-' in list_year:
                        dict_vehicle[model][engine] += [int(year.strip()) for year in list_year.split('-')]
                    else:
                        dict_vehicle[model][engine] += [int(list_year)]

            dict_vehicle[model][engine] = sorted(list(set(dict_vehicle[model][engine])))

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for model in dict_vehicle:
        list_year = []
        for engine in dict_vehicle[model]:
            list_year += dict_vehicle[model][engine]
        list_year = sorted(list(set(list_year)))

        year_min = min(list_year)
        year_max = max(list_year)
        if year_min == year_max:
            list_vehicle.append(model + ' ' + str(year_min))
        else:
            list_vehicle.append(model + ' ' + str(year_min) + '-' + str(year_max))

    vehicle_1 = '\n'.join(list_vehicle + year_loss)
    
    # = = = = = = = = = = = = = = =
    
    list_vehicle = []
    for model in dict_vehicle:
        for engine in dict_vehicle[model]:
                year_min = min(dict_vehicle[model][engine])
                year_max = max(dict_vehicle[model][engine])
                if year_min == year_max:
                    if engine == '':
                        list_vehicle.append(model + ' ' + str(year_min))
                    else:
                        list_vehicle.append(model + ' ' + str(year_min) + ' ' + engine)
                else:
                    if engine == '':
                        list_vehicle.append(model + ' ' + str(year_min) + '-' + str(year_max))
                    else:
                        list_vehicle.append(model + ' ' + str(year_min) + '-' + str(year_max) + ' ' + engine)

    vehicle_2 = '\n'.join(list_vehicle + year_loss)

    # = = = = = = = = = = = = = = =
    
    df_temp = pd.DataFrame([{'序号': int(list_file[a].replace('.txt', '')),
                             'Vehicle_1': vehicle_1,
                             'Vehicle_2': vehicle_2}])
  
    df = pd.concat([df, df_temp], ignore_index=True).fillna('')

    # = = = = = = = = = = = = = = =
    
df = df.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
df.to_excel('./vehicle.xlsx', index=False)

df

Progress: 100%|███████████████████████████| 771/771 [00:00<00:00, 858.46it/s]


,序号,Vehicle_1,Vehicle_2
0,1,NYA,NYA
1,2,C (202) 1993-2001\nC (203) 2000-2008\nC (204) ...,C (202) 1993-2001 C 180 (1.8) - 89 kW / 121 hp...
2,3,C (202) 1993-2001\nC (203) 2000-2008\nC (204) ...,C (202) 1993-2001 C 180 (1.8) - 89 kW / 121 hp...
3,4,C (202) 1993-2001\nC (203) 2000-2008\nC (204) ...,C (202) 1993-2001 C 180 (1.8) - 89 kW / 121 hp...
4,5,C (202) 1993-2001\nC (203) 2000-2008\nC (204) ...,C (202) 1993-2001 C 180 (1.8) - 89 kW / 121 hp...
...,...,...,...
766,767,NYA,NYA
767,768,CR-V II 4WD (RD5) 2001-2007\nCR-V II 4WD (RD8)...,CR-V II 4WD (RD5) 2001-2007 2.0 16V - 110 kW /...
768,769,308 II 2009-2021,308 II 2009-2021 1.6 GTI 16V - 184 kW / 250 hp...
769,770,Sorento I 4x4 2002-2011,Sorento I 4x4 2002-2011 2.5 CRDi 16V - 103 kW ...


In [6]:
df_part = pd.read_excel('./part.xlsx',
                        header=0,
                        dtype=str).fillna('')

df_part['序号'] = df_part['序号'].astype(int)

# = = = = = = = = = = = = = = =

df = pd.merge(left=df_part,
              right=df,
              how='left',
              left_on='序号',
              right_on='序号')

# = = = = = = = = = = = = = = =

df['Vehicle_1'] = ['\n'.join([make + ' ' + vehicle for vehicle in vehicle_1.split('\n')]) for make, vehicle_1 in zip(df['list_make'].tolist(), df['Vehicle_1'].tolist())]
df['Vehicle_2'] = ['\n'.join([make + ' ' + vehicle for vehicle in vehicle_2.split('\n')]) for make, vehicle_2 in zip(df['list_make'].tolist(), df['Vehicle_2'].tolist())]

# = = = = = = = = = = = = = = =

df['序号'] = ''
list_column = df.columns.tolist()
list_column.remove('Make')
list_column.remove('list_make')
list_column.remove('Vehicle_1')
list_column.remove('Vehicle_2')

df_temp = df[list_column].drop_duplicates(ignore_index=True)
df_temp['序号'] = [i+1 for i in range(len(df_temp))]

# = = = = = = = = = = = = = = =

df_temp_2 = df[['Part_Number', 'Vehicle_1', 'Vehicle_2']]

df_temp_2 = df_temp_2.groupby([df_temp_2['Part_Number']], as_index=False).agg({'Vehicle_1': '\n'.join, 'Vehicle_2': '\n'.join})

# = = = = = = = = = = = = = = =

df = pd.merge(left=df_temp,
              right=df_temp_2,
              how='left',
              left_on='Part_Number',
              right_on='Part_Number')

# = = = = = = = = = = = = = = =

df.to_excel('./result.xlsx', index=False)

df

,序号,Brand,Part_Number,Title,Subtitle,Pic,Url,Src,Brand_Id,Master_Id,...,OE_Jaguar,OE_Land Rover,no. of connections,OE_Alpina (BMW),OE_DS,height,pitch circle diameter,no. of holes,Vehicle_1,Vehicle_2
0,1,febi,38872,Axial Bearing for automatic transmission,,,https://partsfinder.bilsteingroup.com/en/artic...,https://cdn.partsfinder.bilsteingroup.com/pf-a...,6101192,1489386,...,,,,,,,,,Chrysler NYA\nMercedes-Benz C (202) 1993-2001\...,Chrysler NYA\nMercedes-Benz C (202) 1993-2001 ...
1,2,febi,38871,Axial Bearing for automatic transmission,,,https://partsfinder.bilsteingroup.com/en/artic...,https://cdn.partsfinder.bilsteingroup.com/pf-a...,6102177,1723728,...,,,,,,,,,Mercedes-Benz C (202) 1993-2001\nMercedes-Benz...,Mercedes-Benz C (202) 1993-2001 C 180 (1.8) - ...
2,3,febi,38869,Axial Bearing for automatic transmission,,,https://partsfinder.bilsteingroup.com/en/artic...,https://cdn.partsfinder.bilsteingroup.com/pf-a...,6102726,1723736,...,,,,,,,,,Mercedes-Benz C (202) 1993-2001\nMercedes-Benz...,Mercedes-Benz C (202) 1993-2001 C 180 (1.8) - ...
3,4,febi,38870,Axial Bearing for automatic transmission,,,https://partsfinder.bilsteingroup.com/en/artic...,https://cdn.partsfinder.bilsteingroup.com/pf-a...,6102737,1723733,...,,,,,,,,,Mercedes-Benz C (202) 1993-2001\nMercedes-Benz...,Mercedes-Benz C (202) 1993-2001 C 180 (1.8) - ...
4,5,febi,33460,Axle Beam Mount for axle support and transmiss...,Fitting Position: vehicle-dependent installati...,,https://partsfinder.bilsteingroup.com/en/artic...,https://cdn.partsfinder.bilsteingroup.com/pf-a...,4885807,383720,...,,,,,,,,,BMW 3 (E36) 1990-2001\nBMW 5 (E39) 1995-2004\n...,BMW 3 (E36) 1990-2001 316i - 73 kW / 99 hp\nBM...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,421,febi,184934,Transmission Mount,Fitting Position: left,,https://partsfinder.bilsteingroup.com/en/artic...,https://cdn.partsfinder.bilsteingroup.com/pf-a...,31096806,3624555,...,,,,,,,,,Opel Corsa E 2014\nVauxhall NYA,Opel Corsa E 2014 1.3 CDTI 16V - 55 kW / 75 hp...
421,422,febi,103234,Transmission Mount,Fitting Position: left,,https://partsfinder.bilsteingroup.com/en/artic...,https://cdn.partsfinder.bilsteingroup.com/pf-a...,20651715,1608156,...,,,,,,,,,Honda CR-V II 4WD (RD5) 2001-2007\nHonda CR-V ...,Honda CR-V II 4WD (RD5) 2001-2007 2.0 16V - 11...
422,423,febi,104384,Transmission Mount,Fitting Position: left,,https://partsfinder.bilsteingroup.com/en/artic...,https://cdn.partsfinder.bilsteingroup.com/pf-a...,21018557,3548948,...,,,,,,,,,Peugeot 308 II 2009-2021,Peugeot 308 II 2009-2021 1.6 GTI 16V - 184 kW ...
423,424,febi,176492,Transmission Mount,Fitting Position: rear,,https://partsfinder.bilsteingroup.com/en/artic...,https://cdn.partsfinder.bilsteingroup.com/pf-a...,26707352,3505400,...,,,,,,,,,KIA Sorento I 4x4 2002-2011,KIA Sorento I 4x4 2002-2011 2.5 CRDi 16V - 103...
